In [10]:
from engine.torchdata import load_file, process_expression, process_json
from engine.torchengine import AnalyticalSetSympy
import torch
import sympy as sp

In [2]:
disciplines = ['pearl_geom', 'pearl_hydro', 'pearl_mass', 'pearl_prop', 'pearl_power', 'pearl_solar', 'pearl_battery','pearl_prob']

In [3]:
symb_str_mapping = {}
all_analyses = {}
flattened_output = []
equality_constraints_sympy = []
inequality_constraints_sympy = []
for file_name in disciplines:
    data = load_file(file_name)
    equality_constraints_sympy += [
        process_expression(elt, symb_str_mapping) 
        for elt in data.get('equality_constraints',[])]
    inequality_constraints_sympy += [
        process_expression(elt, symb_str_mapping) 
        for elt in data.get('inequality_constraints',[])]
    objective = data.get('objective',None)
    if objective is not None:
        objective = process_expression(objective, symb_str_mapping)
    functional_sets = data.get('functional_sets',[])
    flattened_output += functional_sets
    analysismap, symb_str_mapping = process_json(
        functional_sets, symb_str_mapping)
    all_analyses[file_name] = analysismap

In [4]:
idxrev = {i: elt for i, elt in 
            enumerate(symb_str_mapping.values())}
indices = {elt: torch.tensor([int(i)]) for i, elt in 
            idxrev.items()}
sets ={}
user_groups = {}
# flat sets:
idx = 0
for file_name, analysismap in all_analyses.items():
        user_groups[file_name] = list()
        for _,(analysis,outputvar,residual) in enumerate(analysismap):
                user_groups[file_name].append(idx)
                sets[idx] = AnalyticalSetSympy(analysis, 
                        outputvar=outputvar, indices=indices) 
                idx+=1

In [5]:
from graph.graphutils import all_edges, all_variables
from graph.operators import invert_edges
from presolver.tearing import min_arc_set_assign

In [6]:
p = load_file('pearl_params')
edges_in = {idx: tuple(str(idxrev[elt.item()]) for elt in s.analysis.structure[0] if str(idxrev[elt.item()]) not in p.keys()) for idx,s in sets.items()}
edges_out = {idx: tuple(str(idxrev[elt.item()]) for elt in s.analysis.structure[1] if str(idxrev[elt.item()]) not in p.keys()) for idx,s in sets.items()}
edges = (edges_in, edges_out)

In [7]:
edges_for_solving = invert_edges(*edges)
eqnidxs = list(edges[0].keys())
varidxs = all_variables(*edges_for_solving)
edges_left_right = all_edges(*edges_for_solving)

In [8]:
xsol, model = min_arc_set_assign(edges_left_right, varidxs, eqnidxs, not_output=['D_f','D_d','D_s', 'v'])
outset_minassign = dict((right, left) for left, right in edges_left_right if xsol[left, right] > 0.5)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter WLSSecret
Set parameter LicenseID to value 2527862
Academic license 2527862 - for non-commercial use only - registered to no___@mit.edu


In [11]:
sets_opt = {idx:s.reassign(sp.Symbol(outset_minassign.get(idx, None)), rational=True) for idx,s in sets.items()}

In [12]:
functional_sets = []
for idx, fset in sets_opt.items():
    functional_sets.append(
        {'residual':str(fset.residualexpr),
         'analysis': str(fset.expression), 
        'functionalvar': str(fset.outputvar)})
output = {'functional_sets':functional_sets}

In [13]:
# save json data
import json
with open('../applications/data/pearl_restructured.json', 'w') as f:
    json.dump(output, f, indent=4)